# LangChain: Understanding Memory

In [1]:
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from dotenv import load_dotenv
import os

load_dotenv()

def create_basic_chain():
    chatModel = ChatOpenAI(temperature=0.7, verbose=True)
    def processMessage(userQuestion : str):
        sysMessage = SystemMessage(content='''
        You are a helpful assistant! Your name is Bob. Follow this structure for explanations:
        1) First, let's understand the components
        2) Then, let's explore how they work together
        3) Next, let's look at the science behind it
        4) Finally, let's consider the psychological factors
        ''')
    
        humanMessage = HumanMessage(content=userQuestion)
        return chatModel.invoke([sysMessage, humanMessage])
    return processMessage

## 2. Adding Short-Term Memory


In [ ]:
from langchain.memory import ConversationBufferMemory

def create_chain_with_memory():
    chatModel = ChatOpenAI(temperature=0.7, verbose=True)
    memory = ConversationBufferMemory(
        return_messages=True,  # Returns messages in a structured format
        memory_key="chat_history"  # Where to store the memory
    )
    
    def processMessage(userQuestion: str):
        # Get previous chat history
        chat_history = memory.chat_memory.messages
        
        # Create system message
        sysMessage = SystemMessage(content='''
        You are a helpful assistant! Your name is Bob. Follow this structure for explanations:
        1) First, let's understand the components
        2) Then, let's explore how they work together
        3) Next, let's look at the science behind it
        4) Finally, let's consider the psychological factors
        ''')
        
        # Add new human message
        humanMessage = HumanMessage(content=userQuestion)
        
        # Combine all messages
        messages = [sysMessage] + chat_history + [humanMessage]
        
        # Get response
        response = chatModel.invoke(messages)
        
        # Save the interaction to memory
        memory.chat_memory.add_user_message(userQuestion)
        memory.chat_memory.add_ai_message(response.content)
        
        return response
    
    return processMessage

chain = create_chain_with_memory()

response1 = chain("Hi Bob, my name is Starship commander")
print(response1.content)

response2 = chain("What's my name?")
print(response2.content)

## 3. Managing Long Conversation History

In [ ]:
def create_chain_with_managed_memory():
    chatModel = ChatOpenAI(temperature=0.7, verbose=True)
    
    # Initialize memory with a limit
    memory = ConversationBufferMemory(
        return_messages=True,
        memory_key="chat_history",
        k=5  # Keep only last 5 interactions
    )
    
    def processMessage(userQuestion: str):
        # Get managed chat history
        chat_history = memory.chat_memory.messages[-5:]  # Only last 5 messages
        
        sysMessage = SystemMessage(content='''
        You are a helpful assistant! Your name is Bob. Follow this structure for explanations:
        1) First, let's understand the components
        2) Then, let's explore how they work together
        3) Next, let's look at the science behind it
        4) Finally, let's consider the psychological factors
        ''')
        
        humanMessage = HumanMessage(content=userQuestion)
        
        # Combine all messages
        messages = [sysMessage] + chat_history + [humanMessage]
        
        # Get response
        response = chatModel.invoke(messages)
        
        # Save to memory (it will automatically keep only last 5 due to k=5)
        memory.chat_memory.add_user_message(userQuestion)
        memory.chat_memory.add_ai_message(response.content)
        
        # Optional: Print current memory state
        print("\nCurrent Memory State:")
        print(f"Number of messages in memory: {len(memory.chat_memory.messages)}")
        
        return response

    return processMessage

def main():
    chain = create_chain_with_managed_memory()
    
    print("Welcome to the Memory-Managed Chat System!")
    print("Type 'quit' to exit\n")
    
    while True:
        user_question = input("\nYou: ")
        
        if user_question.lower() == 'quit':
            break
            
        try:
            response = chain(user_question)
            print("\nBob:", response.content)
            
        except Exception as e:
            print(f"\nAn error occurred: {str(e)}")
            print("Please make sure your OpenAI API key is set correctly.")

if __name__ == "__main__":
    if not os.getenv("OPENAI_API_KEY"):
        print("Please set your OPENAI_API_KEY environment variable")
    else:
        main()

## 4. Using Summary Memory

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage
from langchain.memory import ConversationSummaryMemory
from dotenv import load_dotenv
import os

load_dotenv()

def create_chain_with_summary_memory():
    chatModel = ChatOpenAI(temperature=0.7, verbose=True)
    
    # Initialize summary memory
    memory = ConversationSummaryMemory(
        llm=chatModel,
        return_messages=True,
        memory_key="chat_history"
    )
    
    def processMessage(userQuestion: str):
        # Get the current chat history
        chat_history = memory.chat_memory.messages
        
        sysMessage = SystemMessage(content='''
        You are a helpful assistant! Your name is Bob. Follow this structure for explanations:
        1) First, let's understand the components
        2) Then, let's explore how they work together
        3) Next, let's look at the science behind it
        4) Finally, let's consider the psychological factors
        ''')
        
        humanMessage = HumanMessage(content=userQuestion)
        messages = [sysMessage] + chat_history + [humanMessage]
        
        # Get response
        response = chatModel.invoke(messages)
        
        # Add to memory
        memory.chat_memory.add_user_message(userQuestion)
        memory.chat_memory.add_ai_message(response.content)
        
        # Print current memory state
        print("\nCurrent Memory State:")
        print(f"Number of messages: {len(memory.chat_memory.messages)}")
        
        # If conversation is long, show the current summary
        if len(memory.chat_memory.messages) > 4:
            print("\nCurrent Conversation Summary:")
            print(memory.predict_new_summary(
                messages=memory.chat_memory.messages,
                existing_summary=""
            ))
        
        return response

    return processMessage

def main():
    chain = create_chain_with_summary_memory()
    
    print("Welcome to the Summary Memory Chat System!")
    print("Type 'quit' to exit\n")
    print("Note: After 4 interactions, you'll see a summary of the conversation\n")
    
    while True:
        user_question = input("\nYou: ")
        
        if user_question.lower() == 'quit':
            break
            
        try:
            response = chain(user_question)
            print("\nBob:", response.content)
            
        except Exception as e:
            print(f"\nAn error occurred: {str(e)}")
            print("Please make sure your OpenAI API key is set correctly.")

if __name__ == "__main__":
    if not os.getenv("OPENAI_API_KEY"):
        print("Please set your OPENAI_API_KEY environment variable")
    else:
        main()

## 5. Testing Different Memory Types

In [ ]:
# Test conversation sequence
def test_memory_chain(chain, name):
    print(f"\nTesting {name}:")
    
    questions = [
        "Hi Bob, my name is Alex",
        "What's my name?",
        "Can you explain what AI is?",
        "What did you just tell me about AI?",
        "How does this relate to machine learning?"
    ]
    
    for i, question in enumerate(questions, 1):
        print(f"\nInteraction {i}:")
        print(f"Human: {question}")
        response = chain(question)
        print(f"AI: {response.content}")

# Test each type of chain
basic_chain = create_basic_chain()
memory_chain = create_chain_with_memory()
managed_chain = create_chain_with_managed_memory()
summary_chain = create_chain_with_summary_memory()

test_memory_chain(basic_chain, "Basic Chain (No Memory)")
test_memory_chain(memory_chain, "Memory Chain")
test_memory_chain(managed_chain, "Managed Memory Chain")
test_memory_chain(summary_chain, "Summary Memory Chain")